In [ ]:
import numpy as np
import pandas as pd
import matplotlib as plt
import torch 
import torch.nn as nn
import yfinance as yf
import keras 
import os
import tensorflow
import datetime 
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Dense
from keras.layers import TimeDistributed
from keras.layers import Activation
from keras import optimizers
from keras.models import Model
from keras.layers import Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam

: 

In [ ]:
def TickerIntoTensor(ticker='aapl',daysago=100,daysforward=10,split_percentage=0.9):
    df = yf.Ticker(ticker.upper())
    df = df.history(period='max')
    if df.shape[1] == 7:
        df = df.drop(columns=['Dividends','Stock Splits'])
    df = df.drop(columns=['Open','High','Low','Volume'])
    for i in range(daysago,-daysforward-1,-1):
        df['{}daysago'.format(i)] = df['Close'].shift(i)
        df = df.copy()
    df = df.drop(columns=['Close'])
    df = df.drop(index=df.tail(daysforward).index)
    df = df.drop(index=df.head(daysago).index)
    sc = MinMaxScaler(feature_range=(0,1))
    df_scaled = sc.fit_transform(df)
    X,Y = df_scaled[:,:daysago+1],df_scaled[:,-daysforward:]
    split = int(len(X)*split_percentage)
    X_train, X_test, Y_train, Y_test = X[:split], X[split:], Y[:split], Y[split:]
    X_train, X_test, Y_train, Y_test = X_train.reshape((-1,daysago+1,1)), X_test.reshape((-1,daysago+1,1)), Y_train.reshape((-1,daysforward,1)), Y_test.reshape((-1,daysforward,1))
    #X_train, X_test, Y_train, Y_test = torch.tensor(X_train).float(), torch.tensor(X_test).float(), torch.tensor(Y_train).float(), torch.tensor(Y_test).float()
    return X_train, X_test, Y_train, Y_test
TickerIntoTensor()

: 

In [ ]:
ticker='aapl'
daysago=100
daysforward=1
split_percentage=0.9
df = yf.Ticker(ticker.upper())
df = df.history(period='max')
if df.shape[1] == 7:
    df = df.drop(columns=['Dividends','Stock Splits'])
df = df.drop(columns=['Open','High','Low','Volume'])
for i in range(daysago,-daysforward-1,-1):
    df['{}daysago'.format(i)] = df['Close'].shift(i)
    df = df.copy()
df = df.drop(columns=['Close'])
df = df.drop(index=df.tail(daysforward).index)
df = df.drop(index=df.head(daysago).index)
print(df.shape[1] == daysago+1+daysforward)
sc = MinMaxScaler(feature_range=(0,1))
df_scaled = sc.fit_transform(df)
X,Y = df_scaled[:,:daysago+1],df_scaled[:,-daysforward:]
print(Y.shape)
split = int(len(X)*split_percentage)
X_train, X_test, Y_train, Y_test = X[:split], X[split:], Y[:split], Y[split:]
X_train, X_test, Y_train, Y_test = X_train.reshape((-1,daysago+1,1)), X_test.reshape((-1,daysago+1,1)), Y_train.reshape((-1,daysforward,1)), Y_test.reshape((-1,daysforward,1))

print(Y_train.shape)

: 

In [ ]:
model1 = Sequential()
model1.add(InputLayer((101, 1)))
model1.add(LSTM(500))
model1.add(Dense(150, 'sigmoid'))
model1.add(Dense(1, 'linear'))

cp1 = ModelCheckpoint('model/.keras', save_best_only=True)
model1.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.01), metrics=[RootMeanSquaredError()])
model1.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=10, callbacks=[cp1])

: 

In [ ]:
from tensorflow.keras.models import load_model
model1 = load_model('model/.keras')
y_pred = model1.predict(TickerIntoTensor('goog')[1])
Y_test = TickerIntoTensor('goog')[3].reshape(-1,10)
avg_error = (np.sum(np.sqrt((Y_test - y_pred)**2)))/len(Y_test)*100

print(avg_error)
avg_error = (np.sum(y_pred - Y_test))/len(Y_test)*100
print(avg_error)

: 

In [ ]:
y_pred,Y_test = pd.DataFrame(y_pred),pd.DataFrame(Y_test)
for i in range(0,y_pred.shape[1]): 
    oneday= pd.concat([y_pred[i],Y_test[i]],axis=1)
    oneday.plot(figsize=(16,8),color = ('black','green'))
    

: 

In [ ]:
model1.summary()

: 